<center> <h2><b>Analysis notebook</b></h2> </center>

In [82]:
from datetime import datetime, timedelta
from mplsoccer import VerticalPitch
from matplotlib import animation
import matplotlib.pyplot as plt
from pathlib import Path
import imageio.v2 as iio
from tqdm import tqdm
import pandas as pd
import matplotlib
import zipfile
import shutil
import time
import json
import enum

import seaborn as sns
import matplotlib.pyplot as plt


# set pandas option to show columns of a dataframe
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

### Path of file to analyse

In [93]:
file_path = 'input/skillcorner/1381458/1381458_dynamic_events.csv'

### Helpers methods

In [94]:
# Transform the timestamp in numerical value
def transformTimestamp(x):
    x = str(x).split('.')[0].split(':')
    return int(x[0]) * 60 + int(x[1])

# Plot categorical values by frequency
def histogram_with_categorical_values(title, data):
    on_target_count = data.value_counts().reset_index()
    on_target_count.columns = ['On Target', 'Count']

    sns.set_theme(style="darkgrid")
    palette = sns.color_palette("coolwarm", len(on_target_count))

    sns.barplot(x='On Target', y='Count', data=on_target_count, palette=palette)

    plt.title(title)
    plt.xticks(rotation=45)
    plt.show()

#### Possesion duration

In [95]:
dynamic_events = pd.read_csv(file_path)
dynamic_events['time_start'] = dynamic_events['time_start'].apply(transformTimestamp)
dynamic_events['time_end'] = dynamic_events['time_end'].apply(transformTimestamp)

dynamic_events['possession_duration'] = (dynamic_events['time_end'] - dynamic_events['time_start'])

team_possession_time = dynamic_events.groupby('team_shortname')['possession_duration'].sum().reset_index()

total_possession_time = team_possession_time['possession_duration'].sum()

team_possession_time['possession_percentage'] = (team_possession_time['possession_duration'] / total_possession_time) * 100

print(team_possession_time)

  team_shortname  possession_duration  possession_percentage
0     Luxembourg                  668              54.754098
1       Slovakia                  552              45.245902


#### Possesion per player

In [96]:
dynamic_events = pd.read_csv(file_path)
dynamic_events['time_start'] = dynamic_events['time_start'].apply(transformTimestamp)
dynamic_events['time_end'] = dynamic_events['time_end'].apply(transformTimestamp)


dynamic_events['possession_duration'] = (dynamic_events['time_end'] - dynamic_events['time_start'])

player_possession_time = dynamic_events.groupby('player_in_possession_name')['possession_duration'].sum().reset_index()
player_possession_time = player_possession_time.sort_values(by='possession_duration', ascending=False)

print(player_possession_time)

   player_in_possession_name  possession_duration
6               G. Rodrigues                  134
2                  D. Hancko                   92
11                   L. Jans                   78
14                M. Martins                   72
4                   D. Vavro                   70
23                  V. Thill                   64
22                 T. Suslov                   62
12                 M. Chanot                   62
0                   A. Moris                   56
1                 C. Martins                   54
8                   J. Kucka                   53
18                P. Pekarík                   50
3                  D. Sinani                   48
15                 M. Olesen                   46
21                S. Lobotka                   39
20               R. Polievka                   38
10                 L. Gerson                   37
16               M. Škriniar                   36
17                   O. Duda                   31


#### Where do the team plays on the field

In [97]:
third_possession = dynamic_events.groupby(['third_end'])['possession_duration'].sum().reset_index()
total_possession = third_possession['possession_duration'].sum()
third_possession['possession_percentage'] = (third_possession['possession_duration'] / total_possession) * 100
print(third_possession[['third_end', 'possession_percentage']])

         third_end  possession_percentage
0  attacking_third              52.622951
1  defensive_third              11.311475
2     middle_third              36.065574
